In [1]:

# import
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader
from langchain.embeddings import HuggingFaceEmbeddings

from langchain import LLMChain
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate
from langchain.llms.base import LLM
from transformers import AutoTokenizer, AutoModel, AutoConfig
from typing import Any, Dict, List, Mapping, Optional, Tuple, Union
from torch.mps import empty_cache
import torch
from langchain.chains import RetrievalQA


In [2]:
class GLM(LLM):
    max_token: int = 2048
    temperature: float = 0.8
    top_p = 0.9
    tokenizer: object = None
    model: object = None
    history_len: int = 1024
    
    def __init__(self):
        super().__init__()
        
    @property
    def _llm_type(self) -> str:
        return "GLM"
            
    def load_model(self, llm_device="gpu",model_name_or_path=None):
        model_config = AutoConfig.from_pretrained(model_name_or_path, trust_remote_code=True)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,trust_remote_code=True)
        self.model = AutoModel.from_pretrained(model_name_or_path, config=model_config, trust_remote_code=True, device='cuda:3').half() # GLM模块装在gpu: 6



    def _call(self,prompt:str,history:List[str] = [],stop: Optional[List[str]] = None):
        response, _ = self.model.chat(
                    self.tokenizer,prompt,
                    history=history[-self.history_len:] if self.history_len > 0 else [],
                    max_length=self.max_token,temperature=self.temperature,
                    top_p=self.top_p)
        return response

In [3]:
import sys
modelpath = "/data1/dxw_data/llm/chatglm3-6b"
sys.path.append(modelpath)
llm = GLM()
llm.load_model(model_name_or_path = modelpath)
#---------------------------至此, 成功加载模型

2024-06-16 11:55:46.569339: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-16 11:55:46.749517: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-16 11:55:47.531062: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64
2024-06-16 11:55:47.531183: W tensorflow/compiler/xla/stream_exec

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [6]:
# ----------------------------合并回答------------------------ #

In [5]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
import json

# 定义PromptTemplate
prompt = PromptTemplate(
    input_variables=["caption", "comment"],
    template="以下是一个元素的caption和comment，请分别概括说明它们的主题和内容。\n\nCaption: {caption}\n\nComment: {comment}\n\n请概括说明这个元素的主题和内容。"
)

# 创建LLMChain
# llm = OpenAI(api_key="YOUR_API_KEY")
chain = LLMChain(llm=llm, prompt=prompt)

# 读取merged_captions.json文件
with open('/data1/dxw_data/llm/tiktok/merged_captions.json', 'r', encoding='utf-8') as file:
    merged_data = json.load(file)

# 初始化保存结果的字典
summary_data = {}

# 生成每个元素的主题和内容概述
i=0
for key, value in merged_data.items():
    caption = value["caption"]
    comment = value["comment"]
    result = chain.run(caption=caption, comment=comment)
    summary_data[key] = result
    print(result)
    print(f"----------第{i}次完成----------")
    i=i+1

# 输出结果到一个新的JSON文件
summary_data_file_path = "/data1/dxw_data/llm/tiktok/summary_data.json"
with open(summary_data_file_path, 'w', encoding='utf-8') as summary_file:
    json.dump(summary_data, summary_file, ensure_ascii=False, indent=4)

print(f"Summary JSON file saved to: {summary_data_file_path}")


主题：这是一个关于香港迪士尼乐园的冰雪奇缘园区的图片，包含了一位蓝眼睛的女性站在魔法桥上的场景，背景为黑色的魔法桥，场景非常梦幻。图片中还包含了相关文本，如香港迪士尼乐园、艾莎女王、魔雪奇缘世界等。

内容：图片展示了一个类似于冰雪奇缘的场景，一位蓝眼睛的女性站在魔法桥上，桥的背景是黑色的。女性穿着蓝色的裙子，整个场景非常梦幻。图片中还包含了相关的文本信息，如香港迪士尼乐园、艾莎女王、魔雪奇缘世界等，以及一些相关的爆梗，如烟花薯片等。
----------第0次完成----------
Caption的主题是描述一张抖音短视频截图，内容是电动牙刷广告页面，评论区有一条“就一‘刷’难求”的评论。

Comment的主题是描述截图中的文字信息，包括：天猫、徐芬、体芬科技电动牙刷、三维十级调刷、创造健康清洁口腔、智能等。内容主要介绍了该电动牙刷的特点和功能，包括其三维十级调刷、智能等特点，以及消费者对该产品的评价和需求。
----------第1次完成----------
Caption: 这段文字描述了一个关于猫咪的短视频截图，该视频以白色为背景，展示了一只猫咪在床上睡觉的情景。文字提到该视频在评论区获得了很多留言、点赞和播放量，并且还有热搜关于猫咪迷人程度的讨论。

Comment: 这段文字描述了一个关于猫咪的短视频截图，其中包含了一些与猫咪有关的文本，例如@奶芙芙、大清早就在旁边喝喝喝的要饭了、#心态全写在脸上的猫猫#小奶猫等。此外，文字还提到了该视频在评论区获得了很多留言、点赞和播放量，并且还有热搜关于猫咪迷人程度的讨论。
----------第2次完成----------
Caption的主题是“身穿学生装的女生在进行直播”，内容是描述了这个女生正在直播的场景，背景是白色的，手上还戴着蓝色的手表。

Comment的主题是“班级里的某些女43”，内容是关于这个女生在直播中的表现和一些与她相关的信息，包括她手的大小、 someone 肉的称呼、静怡酱的名字等等。具体内容如下：
- 吹你手还挺小的
- @肉脸静怡酱 《班级里的某些女43》#剧情#校园
- 导
- 4.6万
- 1.3万
- 肉脸静怡酱
- 班级里的某些女43
- #剧情#校园
- 不信是吧比啊
- @肉脸静怡酱 《班级里的某些女43》#剧情#校园
- 导
- 4.6万
- 1.3万
- 肉脸静怡酱